In [16]:
# --------------------------------------------------------------
#  HIGH-PRECISION RG: mpmath version (NO singular matrix)
# --------------------------------------------------------------
from mpmath import mp, mpf, matrix, power, log, exp, inf, lu_solve, svd
from itertools import product
from typing import Tuple, List
import numpy as np
import matplotlib.pyplot as plt

mp.dps = 100  # 60 decimal digits precision

# --------------------------------------------------------------
# 1. 3-site cell configurations (global, high-precision)
# --------------------------------------------------------------
ALL_SPINS = list(product([-1, 1], repeat=3))
PLUS_SPINS_LIST  = [s for s in ALL_SPINS if sum(s) >= 1]   # cell spin +1
MINUS_SPINS_LIST = [s for s in ALL_SPINS if sum(s) <= -1]  # cell spin -1

# Convert to mpmath matrices (4 rows x 3 cols)
PLUS_SPINS  = matrix(4, 3)
MINUS_SPINS = matrix(4, 3)

for i, s in enumerate(PLUS_SPINS_LIST):
    for j in range(3):
        PLUS_SPINS[i, j] = mpf(s[j])

for i, s in enumerate(MINUS_SPINS_LIST):
    for j in range(3):
        MINUS_SPINS[i, j] = mpf(s[j])


def vec_mul(a: matrix, b: matrix) -> matrix:
    """Element-wise multiplication of two column vectors"""
    if a.cols != 1 or b.cols != 1 or a.rows != b.rows:
        raise ValueError("Vectors must be column vectors of same length")
    return matrix([[a[i, 0] * b[i, 0]] for i in range(a.rows)])


def intracell_energy(spins: matrix, J2: mpf, J4: mpf) -> matrix:
    """Energy inside one cell: J2*(s0*s1 + s1*s2) + J4*s0*s2"""
    # Extract columns as column vectors
    s0 = matrix([[spins[i, 0]] for i in range(spins.rows)])
    s1 = matrix([[spins[i, 1]] for i in range(spins.rows)])
    s2 = matrix([[spins[i, 2]] for i in range(spins.rows)])
    
    # Element-wise operations
    term1 = vec_mul(s0, s1) + vec_mul(s1, s2)
    term2 = vec_mul(s0, s2)
    return J2 * term1 + J4 * term2


def log_R(spinsL: matrix, EL: matrix, spinsR: matrix, ER: matrix,
          dists: List[Tuple[int, int, int]], Jarr: List[mpf], H: mpf = mpf(0)) -> mpf:
    """log of partial partition function R(s'L,s'R)"""
    nL = spinsL.rows
    nR = spinsR.rows
    terms = []
    for i in range(nL):
        magL = spinsL[i, 0] + spinsL[i, 1] + spinsL[i, 2]
        for j in range(nR):
            magR = spinsR[j, 0] + spinsR[j, 1] + spinsR[j, 2]
            Eint = mpf(0)
            for iL, iR, d in dists:
                if d < len(Jarr):
                    Eint += Jarr[d] * spinsL[i, iL] * spinsR[j, iR]
            terms.append(EL[i, 0] + ER[j, 0] + Eint + H * (magL + magR))
    if not terms:
        return -inf
    mx = max(terms)
    s = sum(exp(t - mx) for t in terms)
    return log(s) + mx


def compute_J_prime(rp: int, Jarr: List[mpf], array_size: int) -> mpf:
    """Renormalized coupling J'(r') — NO distance filtering"""
    start = 3 * rp + 1
    left  = [1, 3, 5]
    right = [start, start + 2, start + 4]

    dists = []
    for iL in range(3):
        for iR in range(3):
            d = abs(right[iR] - left[iL])
            if d == 0: continue
            dists.append((iL, iR, d))

    J2 = Jarr[2] if 2 < len(Jarr) else mpf(0)
    J4 = Jarr[4] if 4 < len(Jarr) else mpf(0)

    ELp = intracell_energy(PLUS_SPINS, J2, J4)
    ELm = intracell_energy(MINUS_SPINS, J2, J4)

    log_pp = log_R(PLUS_SPINS, ELp, PLUS_SPINS, ELp, dists, Jarr, 0)
    log_pm = log_R(PLUS_SPINS, ELp, MINUS_SPINS, ELm, dists, Jarr, 0)

    if log_pp == -inf or log_pm == -inf:
        return mpf(0)
    return mpf('0.5') * (log_pp - log_pm)


def build_recursion_matrix(
    a: mpf,
    J0: mpf,
    max_dist: int,
    array_size: int,
    eps: mpf = mpf('1e-30')
) -> Tuple[matrix, matrix]:
    """Build M_ij = ∂J'_i / ∂J_j  (high precision)"""
    if array_size < 3 * max_dist + 2:
        raise ValueError(f"array_size={array_size} < 3*{max_dist}+2={3*max_dist+2}")

    r = [mpf(i) for i in range(1, array_size + 1)]
    Jvec = [J0 * power(ri, -a) for ri in r]
    Jarr = [mpf(0)] + Jvec

    n = max_dist
    M = matrix(n, n)

    for j in range(n):
        dJ = eps * max(mpf(1), abs(Jarr[j + 1]))
        if dJ == 0: dJ = eps

        Jp = Jarr[:]; Jm = Jarr[:]
        Jp[j + 1] += dJ; Jm[j + 1] -= dJ

        Jp_p = [compute_J_prime(rp, Jp, array_size) for rp in range(1, n + 1)]
        Jp_m = [compute_J_prime(rp, Jm, array_size) for rp in range(1, n + 1)]

        for i in range(n):
            M[i, j] = (Jp_p[i] - Jp_m[i]) / (mpf(2) * dJ)

    return matrix([[x] for x in Jvec[:n]]), M


def solve_linear_system(M: matrix, delta: matrix) -> matrix:
    """δJ = M⁻¹ · δJ'  using mpmath LU solve"""
    try:
        return lu_solve(M, delta)
    except Exception as e:
        print("LU failed, using SVD...")
        U, S, V = svd(M)  # Fixed: use mpmath.svd
        S_inv = matrix(S.rows, S.cols)
        for i in range(min(S.rows, S.cols)):
            S_inv[i, i] = mpf(1) / S[i, i] if abs(S[i, i]) > 1e-40 else mpf(0)
        return V.T * S_inv * U.T * delta


def find_fixed_point_newton(
    a: float,
    J_init: float,
    max_dist: int,
    tol: float = 1e-15,
    max_iter: int = 100
) -> Tuple[List[float], int]:
    """Newton-Raphson with mpmath"""
    a_mp = mpf(str(a))
    J_init_mp = mpf(str(J_init))
    tol_mp = mpf(str(tol))

    N = 3 * max_dist + 2
    r = [mpf(i) for i in range(1, N + 1)]
    J = [J_init_mp * power(ri, -a_mp) for ri in r]
    n = max_dist

    for it in range(max_iter):
        Jarr = [mpf(0)] + J

        J_prime_vals = [compute_J_prime(rp, Jarr, N) for rp in range(1, n + 1)]
        J_prime_vec = matrix([[x] for x in J_prime_vals])

        _, M = build_recursion_matrix(a_mp, mpf(0), max_dist=n, array_size=N)

        J_current = matrix([[x] for x in J[:n]])
        delta = J_prime_vec - J_current

        try:
            dJ = solve_linear_system(M, delta)
            J_new = [J[i] - dJ[i, 0] for i in range(n)]
        except:
            J_new = J[:n]

        if max(abs(J_new[i] - J[i]) for i in range(n)) < tol_mp:
            return [float(x) for x in J_new], it + 1

        J[:n] = J_new

    print("Newton-Raphson did not converge.")
    return [float(x) for x in J[:n]], max_iter

In [17]:
# --------------------------------------------------------------
#  RUN: High-precision fixed point
# --------------------------------------------------------------
a = 1.0
J_init = 0.8
max_dist = 10
N = 3 * max_dist + 2

print(f"max_dist = {max_dist}  →  N = {N}")

J_fixed, n_iter = find_fixed_point_newton(a, J_init, max_dist)

print(f"Converged in {n_iter} steps")
print(f"J*_1 = {J_fixed[0]:.20f}")

# Check rank
_, M = build_recursion_matrix(mpf(a), mpf(0), max_dist, N)
U, S, V = svd(M)  # Fixed: use mpmath.svd
rank = sum(1 for s in S if abs(s) > 1e-40)
print(f"Rank of M: {rank} / {M.cols}")

max_dist = 10  →  N = 32
LU failed, using SVD...
Converged in 1 steps
J*_1 = 0.80000000000000004441
Rank of M: 4 / 10


In [27]:
from mpmath import mp, mpf, matrix, randmatrix, eye, lu_solve, norm

# Set high precision
mp.dps = 10
N = 5

# Create random N×N matrix with high-precision entries
A = randmatrix(N, N, min=0, max=1)


def invert_matrix(A, N):
    # Ensure invertibility (add 1 to diagonal)
    for i in range(N):
        A[i, i] += mpf('1')
    
    # Precompute LU decomposition
    A_lu, p = mp.LU_decomp(A)
    
    # Create inverse matrix
    A_inv = matrix(N, N)
    
    # Solve A @ X = I column by column
    I = eye(N)
    for j in range(N):
        # Extract j-th column of I as an N×1 matrix using slicing
        b = I[:, j]  # This works! mpmath.matrix supports A[:, j]
        x = lu_solve(A, b, _LU=A_lu, _p=p)  # Reuse LU
        A_inv[:, j] = x  # Assign solution to j-th column
    
    return A_inv

A_inv = invert_matrix(A, N)
# Verify: A @ A⁻¹ ≈ I
error = norm(A * A_inv - eye(N))
print(f"\nReconstruction error: {error}")


Reconstruction error: 1.281162149e-14


In [ ]:
# --------------------------------------------------------------
#  newton_rg_mpmath.py
#  Pure-mpmath Newton-Raphson fixed-point finder for the
#  long-range 1-d Ising RG (ferromagnetic case)
# --------------------------------------------------------------

from mpmath import mp, mpf, matrix, eye, norm, lu_solve
from mpmath import log, exp, inf, nstr
from itertools import product

# ------------------------------------------------------------------
#  Precision
# ------------------------------------------------------------------
mp.dps = 100                                   # enough for the whole calculation

# ------------------------------------------------------------------
#  Spin configurations (global, read-only)
# ------------------------------------------------------------------
all_spins = list(product([mpf(-1), mpf(1)], repeat=3))
plus_spins  = [s for s in all_spins if sum(s) >= mpf(1)]
minus_spins = [s for s in all_spins if sum(s) <= mpf(-1)]

# ------------------------------------------------------------------
#  Intracell energy (J2 = nearest, J4 = next-nearest)
# ------------------------------------------------------------------
def intracell_energy(spins, J2, J4):
    """Return list of energies for the supplied spin list."""
    return [J2*s0*s1 + J2*s1*s2 + J4*s0*s2
            for s0, s1, s2 in spins]

# ------------------------------------------------------------------
#  log-R for a pair of cells (H included)
# ------------------------------------------------------------------
def logR(spinsL, EL, spinsR, ER, dists, Jvec, H):
    total = mpf(0)
    for iL, sL in enumerate(spinsL):
        magL = sL[0] + sL[1] + sL[2]
        for iR, sR in enumerate(spinsR):
            magR = sR[0] + sR[1] + sR[2]
            inter = mpf(0)
            for i_left, i_right, d in dists:
                inter += Jvec[d] * sL[i_left] * sR[i_right]
            contrib = EL[iL] + ER[iR] + inter + H*(magL + magR)
            total += exp(contrib)
    return log(total) if total > mpf(0) else -inf

# ------------------------------------------------------------------
#  Renormalised coupling J'(r')  (ferromagnetic symmetry)
# ------------------------------------------------------------------
def Jprime(start_int, Jvec):
    """
    start_int – integer index of the first site of the right cell
    Jvec      – matrix of size (md+1) × 1  (md = 3*max_k+10)
    """
    J2 = Jvec[2, 0]          # mpmath matrices are indexed with **integers**
    J4 = Jvec[4, 0]

    left  = [1, 3, 5]
    right = [start_int, start_int+2, start_int+4]
    dists = [(iL, iR, abs(right[iR]-left[iL]))
             for iL in range(3) for iR in range(3)]

    ELp = intracell_energy(plus_spins,  J2, J4)
    ELm = intracell_energy(minus_spins, J2, J4)

    log_pp = logR(plus_spins,  ELp, plus_spins,  ELp, dists, Jvec, mpf(0))
    log_pm = logR(plus_spins,  ELp, minus_spins, ELm, dists, Jvec, mpf(0))

    if log_pp == -inf or log_pm == -inf:
        return inf
    return mpf('0.5') * (log_pp - log_pm)

# ------------------------------------------------------------------
#  Matrix inversion (your routine – unchanged except mpf literals)
# ------------------------------------------------------------------
def invert_matrix(A, N):
    for i in range(N):
        A[i, i] += mpf('1')
    A_lu, p = mp.LU_decomp(A)
    A_inv = matrix(N, N)
    I = eye(N)
    for j in range(N):
        b = I[:, j]
        x = lu_solve(A, b, _LU=A_lu, _p=p)
        A_inv[:, j] = x
    return A_inv

# ------------------------------------------------------------------
#  Jacobian  ∂J'_i / ∂J_j  by central finite difference
# ------------------------------------------------------------------
def jacobian(Jvec_full, max_k, eps=mpf('1e-40')):
    N = max_k
    Jac = matrix(N, N)

    # baseline J'
    Jp_base = [Jprime(3*(k+1)+1, Jvec_full) for k in range(N)]

    for col in range(N):
        Jplus  = Jvec_full.copy()
        Jminus = Jvec_full.copy()
        dJ = eps * max(mpf(1), abs(Jvec_full[col+1, 0]))
        if dJ == mpf(0):
            dJ = eps
        Jplus[col+1, 0]  += dJ
        Jminus[col+1, 0] -= dJ

        Jp_plus  = [Jprime(3*(r+1)+1, Jplus)  for r in range(N)]
        Jp_minus = [Jprime(3*(r+1)+1, Jminus) for r in range(N)]

        for row in range(N):
            Jac[row, col] = (Jp_plus[row] - Jp_minus[row]) / (mpf(2)*dJ)
    return Jac

# ------------------------------------------------------------------
#  Newton-Raphson fixed-point iteration
# ------------------------------------------------------------------
def newton_fixed_point_rg(a, max_k=25, tol=mpf('1e-50'),
                         max_iter=10, initial_guess=None):
    """
    Solve  J* = R(J*)  by Newton:
        J ← J - M⁻¹ (R(J) - J)
    All arguments that must be integers for loops are converted locally.
    """
    N  = int(max_k)                     # loop index
    md = int(mpf(3)*max_k + mpf(10))    # size of the full interaction vector

    # ---- initial guess ------------------------------------------------
    if initial_guess is None:
        J0 = mpf('1.0')
        Jarr = [mpf(0)]*(md+1)
        for r in range(1, md+1):
            Jarr[r] = J0 * mpf(r)**(-a)
        Jvec = matrix(N, 1)
        for i in range(N):
            Jvec[i, 0] = Jarr[i+1]
    else:
        Jvec = initial_guess

    print(f"Newton iteration for a = {nstr(a,6)}, N = {N}")

    for it in range(int(max_iter)):
        # full vector (size md+1) needed for Jprime
        Jfull = matrix(md+1, 1)
        for i in range(1, N+1):
            Jfull[i, 0] = Jvec[i-1, 0]

        # ---- R(J) ----------------------------------------------------
        Jp = matrix(N, 1)
        for k in range(N):
            start = 3*(k+1) + 1                 # integer!
            Jp[k, 0] = Jprime(start, Jfull)

        # ---- residual  Δ = R(J) - J ----------------------------------
        delta = Jp - matrix([[Jvec[i, 0]] for i in range(N)])

        res = norm(delta, p=inf)
        print(f"Iter {it+1:2d}: residual = {nstr(res,12)}")

        if res < tol:
            print(f"converged after {it+1} steps")
            Jfixed = matrix(md+1, 1)
            for i in range(1, N+1):
                Jfixed[i, 0] = Jvec[i-1, 0]
            return Jfixed, True

        # ---- Jacobian M -----------------------------------------------
        M = jacobian(Jfull, N)

        # ---- invert M -------------------------------------------------
        M_inv = invert_matrix(M, N)

        # ---- Newton correction ----------------------------------------
        corr = M_inv * delta
        Jvec = Jvec - corr

    print("did NOT converge")
    return None, False

# ------------------------------------------------------------------
#  Critical exponents from the fixed point
# ------------------------------------------------------------------
def field_derivative_at_fixed(Jfixed):
    """dH'/dH at H = 0 using central difference."""
    J2 = Jfixed[2, 0]
    J4 = Jfixed[4, 0]
    ELp = intracell_energy(plus_spins, J2, J4)

    # geometry for the nearest-neighbour pair (r'=1)
    left  = [1, 3, 5]
    right = [4, 6, 8]
    dists = [(iL, iR, abs(right[iR]-left[iL]))
             for iL in range(3) for iR in range(3)]

    eps = mpf('1e-40')
    log_pos = logR(plus_spins, ELp, plus_spins, ELp, dists, Jfixed,  eps)
    log_neg = logR(plus_spins, ELp, plus_spins, ELp, dists, Jfixed, -eps)
    return (log_pos - log_neg) / (mpf(2)*eps)

def exponents_from_fixed(Jfixed, a, max_k):
    N = int(max_k)

    # ---- thermal eigenvalue y_T ------------------------------------
    Jac = jacobian(Jfixed, N)
    evs = mp.eig(Jac)[0]
    real_evs = [abs(e) for e in evs if mp.isreal(e)]
    lam_T = max(real_evs)
    y_T = log(lam_T) / log(mpf(3))

    # ---- magnetic eigenvalue y_H -----------------------------------
    dHdH = field_derivative_at_fixed(Jfixed)
    y_H = log(dHdH) / log(mpf(3))

    d = mpf(1)

    nu     = mpf(1) / y_T
    alpha  = mpf(2) - d/y_T
    beta   = (d - y_H) / y_T
    gamma  = (mpf(2)*y_H - d) / y_T
    delta  = y_H / (d - y_H)
    eta    = mpf(3) - y_H

    return {
        'a'     : a,
        'y_T'   : y_T,
        'y_H'   : y_H,
        'nu'    : nu,
        'alpha' : alpha,
        'beta'  : beta,
        'gamma' : gamma,
        'delta' : delta,
        'eta'   : eta,
    }

In [ ]:
a = mpf('1.5')
N = mpf(100)
Jfix, ok = newton_fixed_point_rg(a, max_k=N, tol=mpf('1e-5'))